In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.32.116.26:8470']


In [ ]:
import re
import numpy as np
from matplotlib import pyplot as plt
import os

print(os.getcwd())

/content


In [1]:
!git clone https://github.com/rahulvigneswaran/Intrusion-Detection-Systems.git

Cloning into 'Intrusion-Detection-Systems'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 13.30 MiB | 5.22 MiB/s, done.
Resolving deltas: 100% (91/91), done.
Updating files: 100% (68/68), done.


In [2]:
!git clone https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets.git

Cloning into 'ADRepository-Anomaly-detection-datasets'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 154 (delta 62), reused 122 (delta 47), pack-reused 0
Receiving objects: 100% (154/154), 41.04 MiB | 8.94 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (27/27), done.


In [4]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('ADRepository-Anomaly-detection-datasets\\numerical data\DevNet datasets\celeba_baldvsnonbald_normalised.csv', header=None)
testdata = pd.read_csv('Intrusion-Detection-Systems\dnn\kdd\\binary\Testing.csv', header=None)


# labeldict = {'normal':0,
#              'anomaly':1}

traindata = traindata.tail(-1)
# traindata = traindata.drop(columns=[2, 3, 4])
# traindata = traindata[42].map(labeldict)
traindata = np.asarray(traindata).astype("float32")
traindata = pd.DataFrame(traindata)

testdata = testdata.tail(-1)
# testdata = testdata.drop(columns=[2, 3, 4])
# testdata = testdata[42].map(labeldict)
testdata = np.asarray(testdata).astype("float32")
testdata = pd.DataFrame(testdata)



X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

X_train = np.array(trainX)
X_test = np.array(testT)


batch_size = 64

# 1. define the network
model = Sequential()
model.add(Dense(1024,input_dim=39,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn5layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn5layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=1000, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn5layer/dnn5layer_model.hdf5")


C:\Users\Isaac\AppData\Local\Temp\ipykernel_7448\1879176582.py:20: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  traindata = pd.read_csv('ADRepository-Anomaly-detection-datasets\\numerical data\DevNet datasets\celeba_baldvsnonbald_normalised.csv', header=None)


Epoch 1/1000
3166/3166 [==============================] - ETA: 0s - loss: 0.1820 - accuracy: 0.9226
Epoch 1: loss improved from inf to 0.18203, saving model to kddresults/dnn5layer\checkpoint-01.hdf5
3166/3166 [==============================] - 43s 13ms/step - loss: 0.1820 - accuracy: 0.9226
Epoch 2/1000
3164/3166 [============================>.] - ETA: 0s - loss: 0.1747 - accuracy: 0.9255
Epoch 2: loss improved from 0.18203 to 0.17469, saving model to kddresults/dnn5layer\checkpoint-02.hdf5
3166/3166 [==============================] - 42s 13ms/step - loss: 0.1747 - accuracy: 0.9255
Epoch 3/1000
3163/3166 [============================>.] - ETA: 0s - loss: 0.1726 - accuracy: 0.9264
Epoch 3: loss improved from 0.17469 to 0.17261, saving model to kddresults/dnn5layer\checkpoint-03.hdf5
3166/3166 [==============================] - 43s 13ms/step - loss: 0.1726 - accuracy: 0.9265
Epoch 4/1000
3166/3166 [==============================] - ETA: 0s - loss: 0.1716 - accuracy: 0.9269
Epoch 4: loss

In [1]:
!cat /proc/meminfo

MemTotal:       13297192 kB
MemFree:         3114376 kB
MemAvailable:   12405592 kB
Buffers:          279980 kB
Cached:          8960616 kB
SwapCached:            0 kB
Active:           504664 kB
Inactive:        9203900 kB
Active(anon):       1048 kB
Inactive(anon):   454092 kB
Active(file):     503616 kB
Inactive(file):  8749808 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1128 kB
Writeback:             0 kB
AnonPages:        467968 kB
Mapped:           251744 kB
Shmem:              1344 kB
KReclaimable:     363716 kB
Slab:             403116 kB
SReclaimable:     363716 kB
SUnreclaim:        39400 kB
KernelStack:        4144 kB
PageTables:         6080 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648596 kB
Committed_AS:    2638564 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        9028 kB
VmallocChunk:          0 kB
Percpu:          

In [2]:
#A Colab pro environment should have >20Gb of total memory.
from psutil import virtual_memory
colab_pro = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(colab_pro))

if colab_pro < 20:
  print('Not using a high-RAM runtime')
  # train model with lower settings
else:
  print('You are using a high-RAM runtime!')
  # train model with higher settings

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime
